<a href="https://colab.research.google.com/github/liggettla/Malthusian/blob/master/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from pprint import pprint

In [190]:
class Yeast:
    def __init__(self):
        self.o2 = -1
        self.co2 = 1

class Cyanobacteria:
    def __init__(self):
        self.o2 = 1
        self.co2 = -1

def resource_check(individual, global_o2, global_co2):
    if global_o2 + individual.o2 >= 0 and global_co2 + individual.co2 >= 0:
        return True
    else:
        return False

def metabolize(individual, global_o2, global_co2):
    global_o2 += individual.o2
    global_co2 += individual.co2
    return global_o2, global_co2

def reproduce(df, organism, individual):
    pass

def year(df, global_o2, global_co2):
    newborns = []

    # iterate through keys so dict items can be deleted without error
    for organism in list(df.keys()):
        for individual in list(df[organism].keys()):
            # check for sufficient resources
            if resource_check(df[organism][individual], global_o2, global_co2):
                # metabolize
                global_o2, global_co2 = metabolize(df[organism][individual], global_o2, global_co2)
                # reproduce
                reproduce(df, organism, individual)

            # die if insufficient resourses
            else:
                del df[organism][individual]
    
    return global_o2, global_co2

In [191]:
global_o2 = 2
global_co2 = 2

df = {
    'Yeast' : {
        1 : Yeast(),
        2 : Yeast(),
    },

    'Cyanobacteria' : {
        1 : Cyanobacteria()
    }
}

In [193]:
global_o2, global_co2 = year(df, global_o2, global_co2)

print(global_o2, global_co2)
print(df)

1 3
{'Yeast': {1: <__main__.Yeast object at 0x7f4822a9b0d0>}, 'Cyanobacteria': {1: <__main__.Cyanobacteria object at 0x7f4822a7f350>}}
